In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [5]:
arr = np.array([[6, 7, 8], [1, 2, 3], [9, 3, 2]])
print (arr[-1, 0:2])

[9 3]


In [2]:
df = pd.read_csv('data/house_data.csv')

In [3]:
# reading the data
df.info()
df.shape
df.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85509 entries, 0 to 85508
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Price        85509 non-null  object 
 1   Address      85509 non-null  object 
 2   Bedrooms     64999 non-null  object 
 3   Bathrooms    69439 non-null  object 
 4   Size         73698 non-null  object 
 5   Sale Status  69960 non-null  object 
 6   URL          85509 non-null  object 
 7   Raw Price    85509 non-null  float64
dtypes: float64(1), object(7)
memory usage: 5.2+ MB


,Price,Address,Bedrooms,Bathrooms,Size,Sale Status,URL,Raw Price
85504,"$79,000,000","2 Park Pl, New York, NY 10007",NaN,1 ba,"9,680 sqft",Condo for sale,https://www.zillow.com/homedetails/2-Park-Pl-N...,79000000.0
85505,"$90,000,000","432 Park Ave #82, New York, NY 10022",6 bds,8 ba,"8,054 sqft",Condo for sale,https://www.zillow.com/homedetails/432-Park-Av...,90000000.0
85506,"$95,000,000","1441 Angelo Dr, Los Angeles, CA 90210",NaN,NaN,NaN,Lot / Land for sale,https://www.zillow.com/homedetails/1441-Angelo...,95000000.0
85507,"$99,000,000","908 Bel Air Rd, Los Angeles, CA 90077",9 bds,20 ba,"34,000 sqft",House for sale,https://www.zillow.com/homedetails/908-Bel-Air...,99000000.0
85508,"$110,000,000","30 Beverly Park Ter, Beverly Hills, CA 90210",8 bds,12 ba,-- sqft,House for sale,https://www.zillow.com/homedetails/30-Beverly-...,110000000.0


In [ ]:
# drop some columns and clean a little
df.dropna(axis='rows', inplace=True)
df = df[df.Bedrooms != '-- bds']
df = df[df.Bathrooms != '-- ba']
df = df[df.Size != '-- sqft']
df.Price = df['Raw Price']
df.drop(['URL', 'Raw Price'], axis=1, inplace=True)
df.tail()

,Price,Address,Bedrooms,Bathrooms,Size,Sale Status
85501,66000000.0,"111 W 57th St PENTHOUSE 72, New York, NY 10019",4 bds,6 ba,"7,130 sqft",Condo for sale
85502,68000000.0,"0 Del Valle Rd, Livermore, CA 94550",4 bds,2 ba,"2,500 sqft",House for sale
85503,75000000.0,"1060 Brooklawn Dr, Los Angeles, CA 90077",13 bds,17 ba,"15,011 sqft",House for sale
85505,90000000.0,"432 Park Ave #82, New York, NY 10022",6 bds,8 ba,"8,054 sqft",Condo for sale
85507,99000000.0,"908 Bel Air Rd, Los Angeles, CA 90077",9 bds,20 ba,"34,000 sqft",House for sale


In [ ]:
#convert bedrooms into num
bds = list(df.Bedrooms.unique())
bdsnum = [2,3,4,6,7,5,8,11,12,17,9,10,28,16,13,15,14,18,19,36,25,22,21,24,30,23,32,84,35,70,20,78]
df.Bedrooms = df.Bedrooms.map(dict(zip(bds,bdsnum)))


In [ ]:
#convert bedrooms into num in another way

#for position, bedroom_str in zip(df.index, df['Bedrooms']):
  #df['Bedrooms'][position] = bedroom_str[:-4]
  
#df['Bedrooms']

In [ ]:
#df['Bedrooms'] = df['Bedrooms'].astype('int64')
#df.dtypes

In [ ]:
#convert badthroom into num
ba = list(df.Bathrooms.unique())
banum=[]
for i in range(len(ba)):
  banum.append(ba[i].replace(' ba', ''))

for i in range(len(banum)):
  banum[i] = float(banum[i])
df.Bathrooms = df.Bathrooms.map(dict(zip(ba,banum)))




In [ ]:
#convert size to num
sz = list(df.Size.unique())
sznum =[]
for i in range(len(sz)):
  sz[i] = sz[i].replace(' sqft', '')
  sz[i]= sz[i].replace(',','')
  sznum.append(sz[i])
for i in range(len(sznum)):
  sznum[i] = float(sznum[i])
sz = list(df.Size.unique())
df.Size = df.Size.map(dict(zip(sz,sznum)))



In [ ]:
#convert address into columns
list1=[]
for i in df.Address:
  list1.append(i.split(', '))
dfa=pd.DataFrame(list1)
list2=[]
for i in dfa[2]:
  list2.append(str(i).split(' '))
dfa2=pd.DataFrame(list2)
dfa.drop(columns=2,inplace=True)
dfa2.drop(columns=2,inplace=True)


In [ ]:
#reset the original index
df.reset_index(inplace=True)

In [ ]:
#adding Street, City, State, ZipCode into original data
df['Street']=dfa[0]
df['City']=dfa[1]
df['State']=dfa2[0]
df['ZipCode']=dfa2[1]
df.drop(columns='Address',inplace=True)
df.drop(columns='index',inplace=True)


In [ ]:
df.loc[df.ZipCode.isnull()]

,Price,Bedrooms,Bathrooms,Size,Sale Status,Street,City,State,ZipCode
28709,464999.0,2,2.0,1203.0,House for sale,9850 E Mcdowell Mountain Ranch Rd N #1012,NORTH SCOTTSDALE,None,None
28710,464999.0,2,2.0,1203.0,Townhouse for sale,9850 E Mcdowell Mountain Ranch Rd N #1013,NORTH SCOTTSDALE,None,None


In [ ]:
df.loc[28709:28711, 'State']= 'AZ'
df.loc[28709:28711, 'ZipCode']= '85260'

In [ ]:
df.loc[df.ZipCode == 'N9V']

,Price,Bedrooms,Bathrooms,Size,Sale Status,Street,City,State,ZipCode
38379,865000.0,4,4.0,2800.0,House for sale,349 Benson Ct,Amherstburg,ON,N9V


In [ ]:
df.drop(df.iloc[38379].name,inplace=True)

In [ ]:
df.ZipCode = df.ZipCode.astype('int')

In [ ]:
#convert Sale Status into type
hstatus = list(df['Sale Status'].unique())
htype = ['House','Townhouse','Multifamily', 'Condo', 'Others', 'Apartment']
df['Sale Status'] = df['Sale Status'].map(dict(zip(hstatus,htype)))
df.rename(columns={"Sale Status": "Type"}, inplace=True)

In [ ]:
df.drop(['Street'], axis=1, inplace=True)

In [ ]:
df['perSqFt'] = df.Price / df.Size

In [ ]:
#make region states
west = ['CA', 'NV', 'AK', 'WA' , 'OR', 'ID', 'MT', 'WY', 'UT', 'CO', 'AZ', 'NM', 'HI']
midwest = ['ND', 'WI','SD', 'NE', 'KS', 'MN', 'IA', 'MO', 'WI', 'IL', 'IN', 'OH','MI']
north = ['PA', 'NY', 'NH', 'MA', 'CT', 'ME', 'DC', 'NJ', 'RI']
south = ['TX', 'OK', 'AR', 'LA', 'MS', 'AL', 'TN', 'KY', 'WV', 'VA', 'MD', 'DE', 'NC', 'SC', 'GA', 'FL']
newall = west + midwest + north + south
newall.sort()
#check data States
alllist = list(df.State.unique())
alllist.sort()

In [ ]:
df2 = df.copy()


In [ ]:
df2 = df2.reset_index(drop=True)


In [ ]:
df2.head()

,Price,Bedrooms,Bathrooms,Size,Type,City,State,ZipCode,perSqFt
0,1.0,2,1.0,814.0,House,Indianapolis,IN,46217,0.001229
1,775.0,2,1.0,1728.0,House,Indianapolis,IN,46218,0.448495
2,1050.0,3,1.0,1000.0,House,Allentown,PA,18102,1.050000
3,1600.0,3,2.0,1418.0,House,Troy,MI,48083,1.128350
4,1700.0,2,2.0,1060.0,Townhouse,Henderson,NV,89014,1.603774


In [ ]:
df2['Region'] = ''
for i in range(len(df2.State)):
  if df2.State[i] in west:
    df2.Region[i] = 'West'
  elif df2.State[i] in north:
    df2.Region[i] = 'North'
  elif df2.State[i] in midwest:
    df2.Region[i] = 'Midwest'
  elif df2.State[i] in south:
    df2.Region[i] = 'South'
  


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: Sett

In [ ]:
df2

,index,Price,Bedrooms,Bathrooms,Size,Type,City,State,ZipCode,perSqFt,Region
0,0,1.0,2,1.0,814.0,House,Indianapolis,IN,46217,0.001229,Midwest
1,1,775.0,2,1.0,1728.0,House,Indianapolis,IN,46218,0.448495,Midwest
2,2,1050.0,3,1.0,1000.0,House,Allentown,PA,18102,1.050000,North
3,3,1600.0,3,2.0,1418.0,House,Troy,MI,48083,1.128350,Midwest
4,4,1700.0,2,2.0,1060.0,Townhouse,Henderson,NV,89014,1.603774,West
...,...,...,...,...,...,...,...,...,...,...,...
45393,45394,66000000.0,4,6.0,7130.0,Condo,New York,NY,10019,9256.661992,North
45394,45395,68000000.0,4,2.0,2500.0,House,Livermore,CA,94550,27200.000000,West
45395,45396,75000000.0,13,17.0,15011.0,House,Los Angeles,CA,90077,4996.336020,West
45396,45397,90000000.0,6,8.0,8054.0,Condo,New York,NY,10022,11174.571641,North


In [ ]:
sns.h

AttributeError: ignored